In [1]:
#cosas a importar
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pylab as plt    # import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [2]:
# cargar los datos
df = pd.read_csv('../Data/attacks.csv', encoding='latin1')# encoding='latin1' para corregir un error
pd.set_option('display.max_columns', None)# Muestra todas las columnas
df                    

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25718,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25720,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25721,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Funciones de analisis preliminar de columnas:
def preliminar(df, col):
    if col == str:
        df[col] = df[col].str.lstrip()
        df[col] = df[col].str.rstrip()
    print('Type of ', col, ': ', df[col].dtype)
    col = df[col]
    
    if col.dtype == 'object':
        col = col.str.lower().str.rstrip('_').str.lstrip('_')
    
    print('Unique shape is:', col.unique().shape)
    if col.unique().shape[0] < 15:
        print('Unique values are:', col.unique())
    else:
        print('First unique values are:', col.unique()[:5])

    valores_nulos = col.isnull().sum()
    print('NaN values:', valores_nulos)

In [4]:
#guardar las dimensiones originales del dataframe
dim_original = df.shape
df.shape
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
# poner el nombre de todas las columnas en minuscula
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '_').str.rstrip('_')
df.columns


Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_(y/n)', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number_1', 'case_number_2', 'original_order', 'unnamed:_22',
       'unnamed:_23'],
      dtype='object')

In [6]:
#para saber el tipo de datos que manejamos
df.info(memory_usage='deep')# vemos la cantidad de valores nulos por columna
# vemos que algunas columnas no tienen el tipo esperable:
error_type = ['case_number', 'date', 'age', 'time', 'case_number1', 'case_number2']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             8702 non-null   object 
 1   date                    6302 non-null   object 
 2   year                    6300 non-null   float64
 3   type                    6298 non-null   object 
 4   country                 6252 non-null   object 
 5   area                    5847 non-null   object 
 6   location                5762 non-null   object 
 7   activity                5758 non-null   object 
 8   name                    6092 non-null   object 
 9   sex                     5737 non-null   object 
 10  age                     3471 non-null   object 
 11  injury                  6274 non-null   object 
 12  fatal_(y/n)             5763 non-null   object 
 13  time                    2948 non-null   object 
 14  species                 3464 non-null 

In [7]:
#Valores nulos :
df.info(memory_usage='deep')# vemos la cantidad de valores  no nulos por columna

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             8702 non-null   object 
 1   date                    6302 non-null   object 
 2   year                    6300 non-null   float64
 3   type                    6298 non-null   object 
 4   country                 6252 non-null   object 
 5   area                    5847 non-null   object 
 6   location                5762 non-null   object 
 7   activity                5758 non-null   object 
 8   name                    6092 non-null   object 
 9   sex                     5737 non-null   object 
 10  age                     3471 non-null   object 
 11  injury                  6274 non-null   object 
 12  fatal_(y/n)             5763 non-null   object 
 13  time                    2948 non-null   object 
 14  species                 3464 non-null 

In [8]:
#Valores nulos :
valores_nulos = df.isnull().sum().sum()# valores nulos totales en columnas( primer sum()) y en filas(segundo)
valores_nulos # = 488276
nan_cols = df.isna().sum()#miro las columnas con valores nulos #df.isnull().sum()---> Es lo mismo
nan_cols[nan_cols>0] #miro las columnas donde hay valores nulos( si hay valores nulos la suma tiene que ser mayor que 0)
len(nan_cols[nan_cols>0]) # = 24, todas las columnas tienen valores nulos

24

In [9]:
nan_rows = df.isna().sum(axis=1)#miro las filas con valores nulos 
nan_rows[nan_rows > 20] #miro las filas donde hay valores nulos( si hay valores nulos la suma tiene que ser mayor que 0)
len(nan_rows[nan_rows > 20])#  = 19421 filas con valores nulos

19421

In [10]:
#Primera limpia:
df.drop(index=nan_rows[nan_rows > 20].index, inplace=True)#eliminamos todas las filas cuyos valores nulos sean superiores a 20
df.shape , dim_original # Hemos reducido considerablemente el numero de filas al quitar las nulas

((6302, 24), (25723, 24))

In [11]:
#Analizamos las columnas y encontramos relaciones entre ellas:
# las columnas date, case_number, case_number1, case_number2, hablan de fechas(cada una tiene un formato diferente). Se podrian llegar a relacionar con las col 'year' y 'time'
#Las columnas ['country', 'area', 'location'] hablan de localización.
#Las col injury y fatal relacionan el tipo de lesión y si es mortal o no.
#Las col 'href' y 'href_formula' también están relacionadas.
#Las dos ultimas col 'unnamed:_22', 'unnamed:_23' son bastante irrelevantes.

In [12]:
#DEFINICIÓN DEL OBJETIVO: Analizar si los ataques de tiburones se han vuelto más agresivos en los últimos  50 años.(1968-2018)
#Para ello redefiniremos el nivel de injury ( leve, grave, muy grave, mortal) usando las col injury y fatal:
#- Categorizar los datos de la col injury( apoyandonos en fatal)
# Utilizaremos las 4 col relacionadas con la fecha para definir nuestro rango temporal:
#-Eliminaremos aquellas filas que no se encuentren en un rango temporal optimo( por no tener suficientes datos dentro del rang)


In [13]:
#ANALISIS DE LA AGRESIVIDAD:

In [14]:
#Analisis preliminar de 'injury'
df['injury'].unique().shape # = 3738 tipos.Demasidos todavía, definir rango temporal antes
df['injury'].unique()
valores_nulos = df['injury'].isnull().sum() # 28 valores nulos
valores_nulos

28

In [15]:
#Analisis preliminar de 'fatal_(y/n)'
df['fatal_(y/n)'].unique().shape # = 9
df['fatal_(y/n)'].unique() #['N', 'Y', nan, 'M', 'UNKNOWN', '2017', ' N', 'N ', 'y']
valores_nulos = df['fatal_(y/n)'].isnull().sum() # 539 valores nulos
valores_nulos

539

In [16]:
#Limpiar datos de 'fatal_(y/n)':
df['fatal_(y/n)'] = df['fatal_(y/n)'].str.lower().str.rstrip('_').str.lstrip('_')
df['fatal_(y/n)'].unique() #['n', 'y', nan, 'm', 'unknown', '2017'].
fila = df.loc[df['fatal_(y/n)'] == 'm'] #suponemos un error al introducir m = n por cercania en el teclado, Comprobado que la persona no vuelve a aparecer más adelante.
df.loc[285, 'fatal_(y/n)'] = 'n'
fila = df.loc[df['fatal_(y/n)'] == '2017'] #accidente ocurrido en 2012, sin sentido = unknown.
df.loc[786, 'fatal_(y/n)'] = 'unknown'
df.loc[df['fatal_(y/n)'] == 'unknown'].shape # = 72. Volveremos a ellos tras categorizar 'injury'.
df['fatal_(y/n)'].fillna('unknown', inplace = True) # rellenamos los nan con 'unknown'.Volveremos a ellos tras categorizar 'injury'.
df.loc[df['fatal_(y/n)'] == 'unknown'].shape # = 611.
df['fatal_(y/n)'].unique() # ['n', 'y', 'unknown']
df.shape , dim_original

((6302, 24), (25723, 24))

In [17]:
#Limpiar datos de 'injury':
df['injury'] = df['injury'].str.lower().str.rstrip('_').str.lstrip('_')
df.loc[df['fatal_(y/n)'] == 'y', 'injury'] = 'fatal' # para cada valor que sea y(yes) en 'fatal' queremos que injury = 'fatal'
df['injury'].unique().shape # = 3222 tipos diferentes todavía
filt = df['injury'].str.contains('fatal', na=False) # comprobamos que valores contienen 'fatal', 
injury_fatal = df.loc[filt, 'injury']
injury_fatal # = 1447 con fatal en algún punto
filt1 = df['injury'].str.contains('fatal') & df['injury'].str.contains('questionable')
print(df['injury'][5590]) # hay un dato cuestionable -> 5590. No se sabe la especie.Eliminamos fila porque no se sabe si es o no un tiburon
df.drop(index = 5590)
df.loc[df['injury'] == 'fatal'].shape # = 1409 con sólo fatal.Vamos a rellenar todos los que tengan fatal con fatal directamente
df['injury'] = df['injury'].str.replace('.*fatal.*', 'fatal', regex = True)
df.loc[df['injury'] == 'fatal'].shape # = 1447 con 'fatal'.Limpio.
df['injury'].isnull().sum() # = 27. Uno menos que antes
df['injury'].fillna('unknown', inplace = True)
df.loc[df['injury'] == 'unknown'].shape # = 29
df['injury'].isnull().sum() # = 0 . No sabemos que tipo de lesión se produjo. Asi que vamos a eliminar las filas porque no son relevantes para nuestro objeto de estudio
df.drop(df.loc[df['injury'] == 'unknown'].index, inplace = True)
df['injury'].unique().shape # = 3189 tipos difetentes. Complicado aún, vamos a tratar de eliminar las filas que no sepamos si son ataques de tiburón y  que estén fuera de nuestro rango temporal

df.loc[df['fatal_(y/n)'] == 'unknown'].shape # = 531
df.loc[df['injury'] == 'fatal', 'fatal_(y/n)'] = 'y' #limpiamos col 'fatal' con 'injury'
df.loc[df['fatal_(y/n)'] == 'unknown'].shape # = 531 no ha aumentado
df.shape , dim_original

reported fatal, but this is a questionable incident


((6273, 24), (25723, 24))

In [18]:
#Analisis preliminar de 'species':
df['species'].unique().shape                 # = 1547 tipos
df['species'].unique()
valores_nulos = df['species'].isnull().sum() # 2821 valores nulos
valores_nulos

2821

In [19]:
#Limpiar datos de 'species':
# queremos buscar datos en los que no estemos seguros que sea un tiburon. La especie no nos importa
df['species'] = df['species'].str.lower().str.rstrip('_').str.lstrip('_')
df['species'].fillna('unknown', inplace = True)
df['species'].unique().shape # = 1532 tipos
filtro = ~df['species'].str.contains(r'shark|unknown', regex=True)
valores = df[filtro]['species']
valores.shape # = 234 valores que no tienen shark
valores.unique().shape# = 49 valores unicos que no tienen shark, vamos a limpiarlos: si es questionable, invalido, eliminamos fila si incluye los char shar lo cambiamos por shark
valores.unique()
df = df[~df['species'].str.contains('questionable|invalid', case=False, na=False, regex=True)]
df['species'].unique().shape # = 1520 tipos
valores.unique().shape # = 45 valores unicos que no tienen shark
valores.shape # = 65 valores
df['species'] = df['species'].str.replace('.*shar|wobbegong|porbeagle|bronze whaler|hammerhead|spurdog|dogfish|blue pointer|c. .*', 'shark', regex=True) # Sustituimos todos los valores de la columna que contengan *shar* en algún punto por 'shark'
df['species'] = df['species'].str.replace(r'\b\w*shark\w*\b', 'shark')# en el cambio anterior a todos los shark les hemos añadido una k demás, la quitamos
valores.unique().shape # = 19 valores unicos que no tienen shark
valores.shape # = 34 valores en total
valores
new_df = df.loc[df['species'].isin(valores)] # almacenamos los valores que nos quedan en un nuevo dt
new_df.shape# comprobamos esos 34 valores, en los cuales no queda claro si es o no un tiburon
df.drop(new_df.index, inplace = True)#al no estar seguros si se trata o no de un tiburon, eliminamos las filas
df.loc[df['species']== 'unknown'].shape # Recordar que tenemos 2822 filas que no estamos seguros de que se trate de un tiburon: dos opciones eliminamos o sustituimos por shark
df.shape , dim_original

((6059, 24), (25723, 24))

In [20]:
#Analisis de las columna 'year':
preliminar(df, 'year')


Type of  year :  float64
Unique shape is: (242,)
First unique values are: [2018. 2017.   nan 2016. 2015.]
NaN values: 2


In [21]:
#tenemos dos valores nulos vamos a buscarlos
indices_nulos = df[df['year'].isnull()].index
indices_nulos # = 187 y 6079
df.loc[187] # es 2017 lo sustituimos
df.loc[6079] # es 1836 lo sustituimos
df['year'][187] = 2017
df['year'][6079] = 1836
preliminar(df, 'year')

Type of  year :  float64
Unique shape is: (241,)
First unique values are: [2018. 2017. 2016. 2015. 2014.]
NaN values: 0


In [22]:
df['year'] = df['year'].astype(str) #convertivos la columna a string
def limpiar_año(valor):
    patron = r"\d{4}"
    coincidencias = re.findall(patron, valor)
    if coincidencias:
        return coincidencias[0]
    else:
        return valor
df['year'] = df['year'].apply(limpiar_año)
preliminar(df, 'year')
patron = ~df['year'].str.contains(r"\d{4}")
new_df = df[patron]
preliminar(new_df, 'year')# son solo cuatro valores únicos
new_df.shape             # son 123 filas cuyo año no sigue el formato. No queda claro cuando fue, eliminamos por nuestro objeto de estudio
df = df.drop(new_df.index)
preliminar(df, 'year')
df.shape , dim_original


Type of  year :  object
Unique shape is: (241,)
First unique values are: ['2018' '2017' '2016' '2015' '2014']
NaN values: 0
Type of  year :  object
Unique shape is: (4,)
Unique values are: ['500.0' '77.0' '5.0' '0.0']
NaN values: 0
Type of  year :  object
Unique shape is: (237,)
First unique values are: ['2018' '2017' '2016' '2015' '2014']
NaN values: 0


((5936, 24), (25723, 24))

In [23]:
#Nuestro objeto de estudio son los ultimos 50 años hasta 1968. Vamos a verlos
df['year'] = df['year'].astype(int)
new_df = df[df['year'] < 1968]
new_df.shape # 2548 filas antes de 1968, fuera de nuestro rango de estudio. Se van fuera
df = df.drop(new_df.index)
df.shape , dim_original

((3388, 24), (25723, 24))

In [24]:
#Analisis de las columnas relacionadas con la fecha:
df.head()

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
0,2018.06.25,25-Jun-2018,2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"no injury to occupant, outrigger canoe and pad...",n,18h00,shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,minor injury to left thigh,n,14h00 -15h00,unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,injury to left lower leg from surfboard skeg,n,07h45,unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,minor injury to lower leg,n,NaN,shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,lacerations to leg & hand shark provoked incident,n,NaN,"shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [25]:
#Analisis preliminar de 'case_number'
preliminar(df, 'case_number')

Type of  case_number :  object
Unique shape is: (3380,)
First unique values are: ['2018.06.25' '2018.06.18' '2018.06.09' '2018.06.08' '2018.06.04']
NaN values: 0


In [26]:
#Analisis preliminar de 'date'
preliminar(df, 'date')

Type of  date :  object
Unique shape is: (2894,)
First unique values are: ['25-jun-2018' '18-jun-2018' '09-jun-2018' '08-jun-2018' '04-jun-2018']
NaN values: 0


In [27]:
#Analisis preliminar de 'case_number_1'
preliminar(df, 'case_number_1')

Type of  case_number_1 :  object
Unique shape is: (3380,)
First unique values are: ['2018.06.25' '2018.06.18' '2018.06.09' '2018.06.08' '2018.06.04']
NaN values: 0


In [28]:
#Analisis preliminar de 'case_number_2'
preliminar(df, 'case_number_2')

Type of  case_number_2 :  object
Unique shape is: (3380,)
First unique values are: ['2018.06.25' '2018.06.18' '2018.06.09' '2018.06.08' '2018.06.04']
NaN values: 0


In [29]:
# Vamos a convertir todos los valores de cada columna al mismo formato: year_month_day
#case_number: tiene un valor nulo, vamos a corregirlo

"""columna = 'case_number'
indices_nulos = df[df[columna].isnull()].index.tolist()
indices_nulos # el indice erróneo es 5488
df.loc[5488] #Lo buscamos y vemos que podemos completarlo con una de las otras columnas, en este caso case_number_1
df['case_number'][5488] = df['case_number_1'][5488]""" #inecesario al limpiar las filas por el año





"columna = 'case_number'\nindices_nulos = df[df[columna].isnull()].index.tolist()\nindices_nulos # el indice erróneo es 5488\ndf.loc[5488] #Lo buscamos y vemos que podemos completarlo con una de las otras columnas, en este caso case_number_1\ndf['case_number'][5488] = df['case_number_1'][5488]"

In [30]:

def limpiar_valor(valor):
    patron = r"\d{4}\.\d{2}\.\d{2}"
    coincidencias = re.findall(patron, valor)
    if coincidencias:
        return coincidencias[0]
    else:
        return valor
#definimos una funcion que elimine las letras de los valores que sigan el patron dado
df['case_number'] = df['case_number'].apply(limpiar_valor)
cantidad_no_cumplen_patron = df[~df['case_number'].str.contains(r"\d{4}\.\d{2}\.\d{2}")]
cantidad_no_cumplen_patron # comprobamos cuantos valores no siguen el patron: 1 que tiene un guión mal colocado
df['case_number'][390] = df['case_number_1'][390]
df['case_number'][390] # perfecto
df['case_number'] = df['case_number'].str.replace(r"(\d{4})\.(\d{2})\.(\d{2})", r"\3-\2-\1")#ledamos la vuelta
df['case_number_1'] = df['case_number'] 
df['case_number_2'] = df['case_number']
df.tail(25)

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
3463,08-08-1968,08-Aug-1968,1968,Unprovoked,COLUMBIA,Magdalena Department,"Makuaka Caño, Taganga",Dynamite fishing,Abel Mattos Antoniou Vasquez,M,18,lacerations to head,n,10h30,shark,"SoHo.com, 11/19/2-11",1968.08.08-Vasquez.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,08-08-1968,08-08-1968,2840.0,NaN,NaN
3464,27-07-1968,27-Jul-1968,1968,Unprovoked,USA,California,"Bodega Rock, Sonoma County",Free diving,Frank Logan,M,25,major injury to torso,n,11h00,"shark, 4 m [13']","D. Miller & R. Collier; R. Collier, pp.41-42; ...",1968.07.27-FrankLogan_Collier.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,27-07-1968,27-07-1968,2839.0,NaN,NaN
3465,23-07-1968,23-Jul-1968,1968,Unprovoked,USA,Florida,Florida Keys,Wading,H. Meacham,M,33,abrasions on lower leg,n,13h30,unknown,"H.D.Baldridge, SAF Case #1542",1968.07.23-NV-Meacham.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,23-07-1968,23-07-1968,2838.0,NaN,NaN
3466,16-07-1968,16-Jul-1968,1968,Unprovoked,USA,South Carolina,"Stono Inlet, near Charleston, Charleston County",Spearfishing using Scuba,Paul Hughes,M,33,swimfin & knee bitten,n,Afternoon,shark,"H.D.Baldridge, p.284",1968.07.16-Hughes.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,16-07-1968,16-07-1968,2837.0,NaN,NaN
3467,10-07-1968,10-Jul-1968,1968,Unprovoked,BAHAMAS,Great Exuma Island,Children's Bay Cay near Rollerville,Small boat with 2 men onboard hit a submerged ...,male,M,NaN,fatal,y,NaN,unknown,"Reported by F.S.T. Baker to M. Vorenberg, pp.1...",1968.07.10-Bahamas.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,10-07-1968,10-07-1968,2836.0,NaN,NaN
3469,25-06-1968,25-Jun-1968,1968,Unprovoked,BAHAMAS,Eleuthera,Spanish Wells,Spearfishing,Roy Pinder,M,17,"bumped shoulder, bit head",n,NaN,shark,"H.D. Baldridge, p.260; Clark, p.87",1968.06.25-Pinder.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,25-06-1968,25-06-1968,2834.0,NaN,NaN
3470,09-06-1968,09-Jun-1968,1968,Unprovoked,USA,Florida,"Mullet Key, Pinellas County",Clamming,Peter Nash,M,17,lacerations to lower right leg & andkle,n,09h30,unknown,"Evening Indeipendent, 6/10/1968, p.3",1968.06.09-Nash.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,09-06-1968,09-06-1968,2833.0,NaN,NaN
3471,00-06-1968,Jun-1968,1968,Unprovoked,ENGLAND,NaN,NaN,NaN,Roy Cloke,M,NaN,arm severely lacerated,n,NaN,shark,"H.D. Baldridge (1994), SAF Case #1515",1968.06.00-NV-Cloke.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,00-06-1968,00-06-1968,2832.0,NaN,NaN
3472,18-05-1968,18-May-1968,1968,Invalid,TANZANIA,Mafia Island,NaN,Schooner sank during a storm,NaN,NaN,NaN,"6 people rescued, 3 shark-scavenged bodies rec...",unknown,NaN,shark involvement not confirmed,R. Wharton,1968.05.18-MafiaIsland.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,18-05-1968,18-05-1968,2831.0,NaN,NaN
3473,00-05-1968,May-1968,1968,Unprovoked,PAPUA NEW GUINEA,New Ireland Province,NaN,Standing,Tamuk Gilaba,M,17,lower leg severely lacerated,n,NaN,unknown,H.D. Baldridge (1994) SAF Case #1520,1968.05.00-NV-Gilaba.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,00-05-1968,00-05-1968,2830.0,NaN,NaN


In [31]:
#Vamos con la columna 'date':
df['date'] = df['date'].astype(str) #convertivos la columna a string
df['date'] = df['date'].str.lower().str.rstrip('_').str.lstrip('_')
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
meses = {'jan': '01', 'feb': '02', 'mar': '03', 'apr': '04', 'may': '05', 'jun': '06', 'jul': '07', 'aug': '08', 'sep': '09', 'oct': '10', 'nov': '11', 'dec': '12'}
# Recorremos cada valor de la columna 'date'
for index, value in df['date'].items():
    # Verificamos si el valor contiene alguna abreviatura de mes
    if any(month in value for month in months):
        # Sustituimos la abreviatura del mes por el número correspondiente
        for month in months:
            value = value.replace(month, meses[month])
        # Actualizamos el valor en el dataframe
        df.loc[index, 'date'] = value
condition =  ~df['date'].str.contains(r"\d{2}-\d{2}-\d{4}\b")
new_df = df[condition]
new_df.shape # = 232 filas no corresponden al formato, seleccionamos aquellas que tienen una s(ref decada)
nuevos_valores = new_df[new_df['date'].str.contains('s')]# = 13 valores contienen s, y 5 no se refieren a una decada, eliminamos las otras 8 filas por no saber claramente el año
df = df[~df['date'].str.contains(r'\d{4}s')]
df.shape , dim_original

((3380, 24), (25723, 24))

In [32]:
#Vamos con las columnas que no tienen relevancia para nuestro objeto de studio: 
# Queremos quitar valores nulos por 'unknown', poner los str en min y capitalizar todas las palabras
def clean(df, col):
    df[col] = df[col].str.strip()
    df[col] = df[col].fillna('unknown')
    df[col] = df[col].str.lower()
    return df

def capitalize(df,col):
    df[col] = df[col].apply(lambda x: x.title())
    return df



In [33]:
#Vamos con la columna 'type':
preliminar(df, 'type')# solo resulta raro dos valores: questionable boatomg


Type of  type :  object
Unique shape is: (9,)
Unique values are: ['boating' 'unprovoked' 'invalid' 'provoked' 'questionable' 'sea disaster'
 nan 'boat' 'boatomg']
NaN values: 2


In [34]:
clean(df, 'type')
df['type'] = df['type'].replace(['questionable', 'invalid'], 'unknown')
df['type'] = df['type'].replace(['boatomg','boat'], 'boating')
capitalize(df, 'type')

preliminar(df, 'type')


Type of  type :  object
Unique shape is: (5,)
Unique values are: ['boating' 'unprovoked' 'unknown' 'provoked' 'sea disaster']
NaN values: 0


In [35]:
#Vamos con la columna 'country':
preliminar(df, 'country')

Type of  country :  object
Unique shape is: (116,)
First unique values are: ['usa' 'australia' 'mexico' 'brazil' 'south africa']
NaN values: 9


In [36]:
clean(df, 'country')
capitalize(df, 'country')# Terminado


,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
0,25-06-2018,25-06-2018,2018,Boating,Usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"no injury to occupant, outrigger canoe and pad...",n,18h00,shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,25-06-2018,25-06-2018,6303.0,NaN,NaN
1,18-06-2018,18-06-2018,2018,Unprovoked,Usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,minor injury to left thigh,n,14h00 -15h00,unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,18-06-2018,18-06-2018,6302.0,NaN,NaN
2,09-06-2018,09-06-2018,2018,Unknown,Usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,injury to left lower leg from surfboard skeg,n,07h45,unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,09-06-2018,09-06-2018,6301.0,NaN,NaN
3,08-06-2018,08-06-2018,2018,Unprovoked,Australia,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,minor injury to lower leg,n,NaN,shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,08-06-2018,08-06-2018,6300.0,NaN,NaN
4,04-06-2018,04-06-2018,2018,Provoked,Mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,lacerations to leg & hand shark provoked incident,n,NaN,"shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,04-06-2018,04-06-2018,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,02-02-1968,02-02-1968,1968,Unprovoked,Australia,New South Wales,Brunswick Heads,NaN,R. Vidler,NaN,NaN,survived,n,NaN,unknown,"J. Green, p.36",1968.02.02-Vidler.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,02-02-1968,02-02-1968,2817.0,NaN,NaN
3487,00-02-1968,02-1968,1968,Unprovoked,Mozambique,NaN,NaN,Diving,Sergio Peres,NaN,24,no injury,n,NaN,unknown,H.D. Baldridge (1994) SAF Case #1522,1968.02.00-NV-Peres.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,00-02-1968,00-02-1968,2816.0,NaN,NaN
3488,17-01-1968,17-01-1968,1968,Unprovoked,Papua New Guinea,Morobe Province,Finschafen,Swimming,Awin Ieromia,NaN,13,fatal,y,17h30,unknown,H.D. Baldridge (1994) SAF Cases #1519 & #1584,1968.01.17-NV-Ieromia.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,17-01-1968,17-01-1968,2815.0,NaN,NaN
3489,00-00-1968,1968,1968,Unprovoked,Costa Rica,Limón Province,Parismina,Swimming after his canoe capsized,male,M,NaN,fatal,y,NaN,unknown,"Kokomo Tribune, 5/11/1969",1968.00.00.c-Parisima.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,00-00-1968,00-00-1968,2814.0,NaN,NaN


In [37]:
#Vamos con la columna 'area':
preliminar(df, 'area')


Type of  area :  object
Unique shape is: (388,)
First unique values are: ['california' 'georgia' 'hawaii' 'new south wales' 'colima']
NaN values: 146


In [38]:
clean(df, 'area')
capitalize(df, 'area')
df['area'] = df['area'].replace('Unknown', pd.NA).fillna(df['country'])



In [39]:
#Vamos con la columna 'location':
preliminar(df, 'location')

Type of  location :  object
Unique shape is: (2265,)
First unique values are: ['oceanside, san diego county' 'st. simon island, glynn county'
 'habush, oahu' 'arrawarra headland' 'la ticla']
NaN values: 185


In [40]:
clean(df, 'location')
capitalize(df, 'location')
df['location'] = df['location'].replace('Unknown', pd.NA).fillna(df['area'])
#Rellenamos los valores unknown por valores conocidos
df['area'] = df['area'].replace('Unknown', pd.NA).fillna(df['location'])
df['country'] = df['country'].replace('Unknown', pd.NA).fillna(df['location'])


In [41]:
#Vamos con la columna 'activity':
preliminar(df, 'activity')

Type of  activity :  object
Unique shape is: (571,)
First unique values are: ['paddling' 'standing' 'surfing' 'free diving' 'kite surfing']
NaN values: 210


In [42]:
clean(df, 'activity')
df

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
0,25-06-2018,25-06-2018,2018,Boating,Usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57,"no injury to occupant, outrigger canoe and pad...",n,18h00,shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,25-06-2018,25-06-2018,6303.0,NaN,NaN
1,18-06-2018,18-06-2018,2018,Unprovoked,Usa,Georgia,"St. Simon Island, Glynn County",standing,Adyson McNeely,F,11,minor injury to left thigh,n,14h00 -15h00,unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,18-06-2018,18-06-2018,6302.0,NaN,NaN
2,09-06-2018,09-06-2018,2018,Unknown,Usa,Hawaii,"Habush, Oahu",surfing,John Denges,M,48,injury to left lower leg from surfboard skeg,n,07h45,unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,09-06-2018,09-06-2018,6301.0,NaN,NaN
3,08-06-2018,08-06-2018,2018,Unprovoked,Australia,New South Wales,Arrawarra Headland,surfing,male,M,NaN,minor injury to lower leg,n,NaN,shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,08-06-2018,08-06-2018,6300.0,NaN,NaN
4,04-06-2018,04-06-2018,2018,Provoked,Mexico,Colima,La Ticla,free diving,Gustavo Ramos,M,NaN,lacerations to leg & hand shark provoked incident,n,NaN,"shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,04-06-2018,04-06-2018,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,02-02-1968,02-02-1968,1968,Unprovoked,Australia,New South Wales,Brunswick Heads,unknown,R. Vidler,NaN,NaN,survived,n,NaN,unknown,"J. Green, p.36",1968.02.02-Vidler.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,02-02-1968,02-02-1968,2817.0,NaN,NaN
3487,00-02-1968,02-1968,1968,Unprovoked,Mozambique,Mozambique,Mozambique,diving,Sergio Peres,NaN,24,no injury,n,NaN,unknown,H.D. Baldridge (1994) SAF Case #1522,1968.02.00-NV-Peres.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,00-02-1968,00-02-1968,2816.0,NaN,NaN
3488,17-01-1968,17-01-1968,1968,Unprovoked,Papua New Guinea,Morobe Province,Finschafen,swimming,Awin Ieromia,NaN,13,fatal,y,17h30,unknown,H.D. Baldridge (1994) SAF Cases #1519 & #1584,1968.01.17-NV-Ieromia.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,17-01-1968,17-01-1968,2815.0,NaN,NaN
3489,00-00-1968,1968,1968,Unprovoked,Costa Rica,Limón Province,Parismina,swimming after his canoe capsized,male,M,NaN,fatal,y,NaN,unknown,"Kokomo Tribune, 5/11/1969",1968.00.00.c-Parisima.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,00-00-1968,00-00-1968,2814.0,NaN,NaN


In [43]:
sust = ['paddling', 'standing', 'surfing', 'diving', 'fishing', 'swimming']
for v in sust:
    df.loc[df['activity'].str.contains(v, case=False, na=False), 'activity'] = v  # por cada valor en la lista si loc. en la col del df si el valor contiene una palabra de la lista la sustituye por la palabra 
preliminar(df, 'activity')


Type of  activity :  object
Unique shape is: (264,)
First unique values are: ['paddling' 'standing' 'surfing' 'diving' 'swimming']
NaN values: 0


In [44]:
df

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
0,25-06-2018,25-06-2018,2018,Boating,Usa,California,"Oceanside, San Diego County",paddling,Julie Wolfe,F,57,"no injury to occupant, outrigger canoe and pad...",n,18h00,shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,25-06-2018,25-06-2018,6303.0,NaN,NaN
1,18-06-2018,18-06-2018,2018,Unprovoked,Usa,Georgia,"St. Simon Island, Glynn County",standing,Adyson McNeely,F,11,minor injury to left thigh,n,14h00 -15h00,unknown,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,18-06-2018,18-06-2018,6302.0,NaN,NaN
2,09-06-2018,09-06-2018,2018,Unknown,Usa,Hawaii,"Habush, Oahu",surfing,John Denges,M,48,injury to left lower leg from surfboard skeg,n,07h45,unknown,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,09-06-2018,09-06-2018,6301.0,NaN,NaN
3,08-06-2018,08-06-2018,2018,Unprovoked,Australia,New South Wales,Arrawarra Headland,surfing,male,M,NaN,minor injury to lower leg,n,NaN,shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,08-06-2018,08-06-2018,6300.0,NaN,NaN
4,04-06-2018,04-06-2018,2018,Provoked,Mexico,Colima,La Ticla,diving,Gustavo Ramos,M,NaN,lacerations to leg & hand shark provoked incident,n,NaN,"shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,04-06-2018,04-06-2018,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3486,02-02-1968,02-02-1968,1968,Unprovoked,Australia,New South Wales,Brunswick Heads,unknown,R. Vidler,NaN,NaN,survived,n,NaN,unknown,"J. Green, p.36",1968.02.02-Vidler.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,02-02-1968,02-02-1968,2817.0,NaN,NaN
3487,00-02-1968,02-1968,1968,Unprovoked,Mozambique,Mozambique,Mozambique,diving,Sergio Peres,NaN,24,no injury,n,NaN,unknown,H.D. Baldridge (1994) SAF Case #1522,1968.02.00-NV-Peres.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,00-02-1968,00-02-1968,2816.0,NaN,NaN
3488,17-01-1968,17-01-1968,1968,Unprovoked,Papua New Guinea,Morobe Province,Finschafen,swimming,Awin Ieromia,NaN,13,fatal,y,17h30,unknown,H.D. Baldridge (1994) SAF Cases #1519 & #1584,1968.01.17-NV-Ieromia.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,17-01-1968,17-01-1968,2815.0,NaN,NaN
3489,00-00-1968,1968,1968,Unprovoked,Costa Rica,Limón Province,Parismina,swimming,male,M,NaN,fatal,y,NaN,unknown,"Kokomo Tribune, 5/11/1969",1968.00.00.c-Parisima.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,00-00-1968,00-00-1968,2814.0,NaN,NaN


In [45]:
#Volvemos con la columna 'injury':

In [46]:
preliminar(df, 'injury')

Type of  injury :  object
Unique shape is: (1930,)
First unique values are: ['no injury to occupant, outrigger canoe and paddle damaged'
 'minor injury to left thigh'
 'injury to left lower leg from surfboard skeg'
 'minor injury to lower leg'
 'lacerations to leg & hand shark provoked incident']
NaN values: 0


In [47]:
"""for i, v in enumerate(df['injury']):
    if 'no injury' in v:
        df.loc[i, 'injury'] = 'no injury'
    elif 'minor' in v:
        df.loc[i, 'injury'] = 'minor injury'
    elif 'severe' in v:
        df.loc[i, 'injury'] = 'severe injury'
    elif 'cut' in v:
        df.loc[i, 'injury'] = 'severe injury'"""



df.shape

(3380, 24)

In [62]:
df['injury'] = df['injury'].str.replace(' ', '_')
df = df[~df['injury'].str.contains('presumed|scaveng|hoax|not_confirmed|doubt|no_detail|drown|never|unconfirmed|disappear|swordfish|not_recovered|crocodile|unidentified')]
df.shape

(3293, 24)

In [67]:
df['injury'] = np.where(df['injury'].str.contains('no_injury|unhurt|uninjured|no_attack|not_injure|no_inur|neither_were_injured|no_ijnury'), 'no injury',
                               np.where(df['injury'].str.contains('nip|bruise|grazed|minor|shallow|contusion|scratch|superficial|toe|finger'), 'minor injury',
                                        np.where(df['injury'].str.contains('gash|amputat|avuls|sever|cut|major|bit|nipped|mouthed|massive|lost|extensive|serious'), 'severe injury',
                                                 np.where(df['injury'].str.contains('lacera|significant|lacerated|puncture|abrasion|inju'), 'injury' , df['injury'])
                                                )
                                       )
                              )
df['injury'].unique().shape

(75,)

In [68]:
df['injury'].unique()

array(['injury', 'minor injury', 'fatal', 'severe injury',
       'wounds_to_upper_thigh_and_lower_leg',
       'struck_by_fin_on_chest_&_leg', 'arm_grabbed_provoked_incident',
       'right_hand_&_part_of_forearm_removed',
       'inuries_to_lower_left_leg_&_ankle', 'wound_to_right_lower_leg',
       'board_snapped_in_two', 'his_boat_was_holed_by_a_shark',
       'possibly_a_provoked_incident', 'unknown,_but_survived',
       'scrape_to_right_forearm._frennet_says_inflicted_by_a_blue_shark,_but_authorities_question_shark_involvement',
       'torn_shoulder_ligament_as_result_of_collision_with_shark',
       'provoked_incident?__', 'stepped_on_shark_provoked_incident',
       'missing',
       'reports_said_a_shark_attacked_the_dinghy,_but_vickery_said_it_did_not',
       'shark_bumped_right_ankle', 'stingray_envenomation,_not_a_shark',
       'survived',
       'killed_by_a_stingray,_not_a_shark_-_a_tragedy_for_his_family_and_marine_wildlife',
       'non-swimmer_pulled_off_pier&_into

In [51]:
#Después de esto quedan pocas injuries sin catalogar:
#thigh

#'harassed_by_sharks_but_not_injured_by_them,'scrape_to_right_forearm._frennet_says_inflicted_by_a_blue_shark,_but_authorities_question_shark_involvement',
#al no estar seguros se elimina filas



In [53]:
#Vamos con la columna 'name':
preliminar(df, 'name')


Type of  name :  object
Unique shape is: (2852,)
First unique values are: ['julie wolfe' 'adyson\xa0mcneely ' 'john denges' 'male' 'gustavo ramos ']
NaN values: 60


In [54]:
df['name'] = df['name'].fillna('unknown')# rellenamos los nulos

In [55]:
preliminar(df, 'sex')


Type of  sex :  object
Unique shape is: (5,)
Unique values are: ['f' 'm' nan 'm ' 'lli']
NaN values: 218


In [56]:
df['New_Column'] = np.where(df['Gender'] == 'male', 'M',
#                           np.where(df['Gender'] == 'female', 'F', df['New_Column']))

#new_df = df[df['name'].str.contains(r'^[^A-Z]')] # seleccionamos cada valor de la col que no empiece por mayus
#preliminar(new_df, 'name') # vemos que hay algunos male and female, antes de continuar vamos a pasar a la col sex para rellenar
#df['name'] = df['name'].replace(['male', 'invalid'], 'unknown')

SyntaxError: incomplete input (3414348025.py, line 6)

In [ ]:
preliminar(new_df, 'name')
new_df.head(50)

In [ ]:

new_df_2 = pd.DataFrame({'new_col': df['name'].apply(lambda x: assign_gender(x))}) #creamos un df con una columna que contenga los male y female

df['sex'] = np.where(new_df_2['new_col'] == 'male', 'M', np.where(new_df_2['new_col'] == 'female', 'F', df['sex']))
#para cada valor que en el nuevo df tenga male o female rellenamos los valores de la columna sex en df
preliminar(df, 'sex')

In [ ]:
#('male').str.strip()